In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import open3d as o3d
import numpy as np
import math 
import copy

def visualization(data):
    if data is None:
        print('data load faile')
        sys.exit()
    for i in range(0,len(data)):
        vis = o3d.visualization.Visualizer()
        obb = data[i].get_oriented_bounding_box()
        vis.create_window()
        vis.add_geometry(obb)
        vis.add_geometry(data[i])
        o3d.visualization.ViewControl.set_zoom(vis.get_view_control(), 0.8)
        vis.run()


    
def plane_detection():
    ###############1
    data = []
    pcd = o3d.io.read_point_cloud("best/centerslowdata/fragment_014.ply")
    #pcd = o3d.io.read_point_cloud("realsensedata.ply")
    #pcd2 = o3d.io.read_point_cloud("best/ceterslolwlow/fragment_009.ply")
    plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                             ransac_n=3,
                                             num_iterations=1000)


    [a, b, c, d] = plane_model
    print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

    inlier_cloud = pcd.select_by_index(inliers)
    inlier_cloud.paint_uniform_color([1.0, 0, 0])

    outlier_cloud = pcd.select_by_index(inliers, invert=True)

    ###############2
    plane2_model, inliers2 = outlier_cloud.segment_plane(distance_threshold=0.01,
                                                         ransac_n=3,
                                                         num_iterations=1000)
    [a, b, c, d] = plane2_model
    print(f"Plane equation2: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")
    
    inlier2_cloud = outlier_cloud.select_by_index(inliers2)
    inlier2_cloud.paint_uniform_color([0, 1.0, 0])

    outlier2_cloud = outlier_cloud.select_by_index(inliers2, invert=True)

    ################3
    plane3_model, inliers3 = outlier2_cloud.segment_plane(distance_threshold=0.01,
                                                          ransac_n=3,
                                                          num_iterations=1000)
    [a, b, c, d] = plane3_model
    print(f"Plane equation3: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")
    inlier3_cloud = outlier2_cloud.select_by_index(inliers3)
    inlier3_cloud.paint_uniform_color([0, 0, 1.0])
    
    data.append(inlier_cloud)
    data.append(inlier2_cloud)
    data.append(inlier3_cloud)
    
    p1 = np.asarray(data[0].points)
    p2 = np.asarray(data[1].points)
    p3 = np.asarray(data[2].points)
    """
    vis = o3d.visualization.Visualizer()    
    vis.create_window()
    #vis.add_geometry(pcd)
    #vis.add_geometry(inlier_cloud)
    #vis.add_geometry(inlier2_cloud)
    vis.add_geometry(inlier3_cloud)
    
    #vis.add_geometry(pcd2)
    o3d.visualization.ViewControl.set_zoom(vis.get_view_control(), 0.8)
    vis.run()
    """
    
    visualization(data)
    

plane_detection()

Plane equation: 0.46x + -0.52y + 0.72z + -1.45 = 0
Plane equation2: 0.97x + 0.23y + -0.04z + 1.20 = 0
Plane equation3: -0.06x + 0.87y + 0.49z + -1.11 = 0
